In [91]:
import pandas as pd

In [92]:
# ファイルの場所

input_data = 'scoredata.xlsx'
output_data = 'JTS_score_data.xlsx'

In [93]:
# データからの検索をするためのリスト

teams = [
    '金沢学院大学クラブ',
    'アベノジュニアトランポリンクラブ',
    '厚木FUSiONスポーツクラブ',
    '日本体育大学トランポリンクラブ',
    '星稜クラブ',
    'キタイスポーツクラブ'
]

names = [
    'ひかる',
    '裕美',
    '愛菜',
    '沙季',
    'セリナ',
    'ここね',
    '守杏',
    '晴茄',
    '希湖',
    '美咲希'
]

stages = [
    'Qualification',
    'Final'
]

In [94]:
# 各大会のスコアデータの読み込み時に必要なデータのみ抽出

df = pd.read_excel(input_data)
df = df[[
    'Competition',
    'Stage',
    'Surname',
    'Given Name',
    'Representing',
    'Mark',
    'Judge',
    '∑'
]]

In [95]:
# 上記のリストを利用し、クラブ名、名前、ルーティン名で検索して一人を抜き出して df1 に格納
# その後、各ジャッジの点数が列として格納されているため、一旦ジャッジ名と点数のみを取り出して転置し df2 に格納
# さらに、df1 と df2 のインデックスを一旦クリアし結合処理

def main(team, name, stage):
    df1 = df[df['Representing'] == team]
    df1 = df1[df1['Given Name'] == name]
    df1 = df1[df1['Stage'] == stage]
    df1 = df1.iloc[:1, [0, 1, 2, 3, 4, 5]]

    df2= df[df['Representing'] == team]
    df2 = df2[df2['Given Name'] == name]
    df2 = df2[df2['Stage'] == stage]
    df2 = df2.loc[:, ['Judge','∑']]
    df2 = df2.set_index('Judge')
    df2 = df2[0:10].T
    
    df1.reset_index(drop=True, inplace=True)
    df2.reset_index(drop=True, inplace=True)

    df3 = pd.concat([df1, df2], axis=1)
    return df3

In [96]:
# result_df という空のリストを用意
# for文にて、各選手の情報を検索し、main() に渡す
# 取り出された各選手のデータを result_df に格納
# result_df に格納された各選手のデータを結合処理

result_df = []

for team in teams:
    for name in names:
        for stage in stages:
            result_df.append(main(team, name, stage))

result_df = pd.concat(result_df)

In [97]:
# 結合時にできた欠損値を含む行をすべて削除

result_df = result_df.dropna(how='all')

In [98]:
# Excel ファイルに保存

result_df.to_excel(output_data, index=False)